In [ ]:
# standard libraries
import os
import random

# data handling
import pandas as pd
import numpy as np

# torch and dataset utils
import torch
from torch.utils.data import Dataset, DataLoader

# transformers library
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# evaluation and data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)

c:\Users\Kev\Documents\GitHub\bias-detector-en-de\thesis-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Seed


calls Python’s random.seed, NumPy’s np.random.seed and PyTorch’s torch.manual_seed

In [ ]:
seed = 10

def set_seed(seed):
    random.seed(seed)                  # Python built-in RNG
    np.random.seed(seed)               # NumPy RNG
    torch.manual_seed(seed)            # PyTorch RNG (CPU)
    torch.cuda.manual_seed(seed)       # PyTorch RNG (current GPU)
    torch.cuda.manual_seed_all(seed)   # All GPUs (if using multi-GPU)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

# Load Dataset

In [3]:
df = pd.read_csv("datasets/dataset_f.csv")

# Load pre-trained model 

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print device info
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU: None (using CPU)")

## Load Tokenizer

In [ ]:
# model name
model_path = "bert-base-multilingual-cased"

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

## Load Model and Send to Device

In [ ]:
# load model with classification head
model = BertForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    id2label={0: "neutral", 1: "biased"},
    label2id={"neutral": 0, "biased": 1}
)

# move model to device
model.to(device)

CUDA Available: True
GPU: NVIDIA GeForce RTX 3070 Laptop GPU


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

# Set trainable parameters


- "transfer learning". we leave the base model parameters frozen, only train a classification head that we add on top
- might result in rigid model
- unfreeze final four layers, keeping computational cost down but keep flexibility

In [ ]:
# freeze encoder layers 0 to 7
for name, param in model.named_parameters():
    if name.startswith("bert.encoder.layer."):
        layer_num = int(name.split(".")[3])
        if layer_num <= 7:
            param.requires_grad = False

# count and print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params}")

Trainable params: 14767874


# Data pre-processing

- PyTorch models need input data in a specific format
- BiasDataset class turns each row from df into tokenized input tensors for BERT

In [ ]:
# custom dataset for bias detection
class BiasDataset(Dataset):
    # init with dataframe and tokenizer
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    # return number of samples
    def __len__(self):
        return len(self.data)

    # return one encoded sample
    def __getitem__(self, idx):
        english = self.data.iloc[idx]["english"]
        german = self.data.iloc[idx]["german"]
        label = int(self.data.iloc[idx]["label"])

        # tokenize EN-DE sentence pair
        encoded = self.tokenizer(
            text=english,
            text_pair=german,
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )

        # encoded outputs have extra batch dimension, remove it with squeeze(0) to get plain tensors
        item = {key: val.squeeze(0) for key, val in encoded.items()}
        # add label tensor to the dict under key 'labels'
        item["labels"] = torch.tensor(label)
        return item


- tokenizer gives tensors with a first size of 1 (a batch)
- squeeze(0) removes that first size, making single samples

## Train test split

In [ ]:
# split data into train (80%) and temp (20%), keeping label distribution same with stratify
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=seed
)

# split temp into validation (10%) and test (10%) sets, stratified by label
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=seed
)


## Create Dataset Objects

In [ ]:
# create train, validation and test datasets
train_dataset = BiasDataset(train_df, tokenizer)  
val_dataset = BiasDataset(val_df, tokenizer)    
test_dataset = BiasDataset(test_df, tokenizer)  

# Evaluation Metrics Calculation for Classification

The `compute_metrics` function calculates evaluation metrics for classification results.

### Inputs
- `eval_pred`: A tuple containing two elements:
  - `logits`: The raw model outputs (before softmax).
  - `labels`: The true class labels.

### Process
1. Get predicted classes by taking the index with the highest logit value.
2. Calculate precision, recall, F1 score, and support for each class separately.
3. Create a detailed classification report as a formatted string.
4. Compute the confusion matrix showing correct and incorrect predictions per class.
5. Prepare a dictionary of metrics including:
   - Precision, recall, F1, and support for each class.
   - Macro averages (mean) for precision, recall, and F1.
   - Overall accuracy.
6. Print the confusion matrix and classification report.

### Output
- Returns a dictionary with all the above metrics, suitable for logging or further analysis.

### Notes
- `average=None` means metrics are computed separately for each class.
- `zero_division=0` avoids errors if some classes have no predicted samples.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)

    precision, recall, f1, support = precision_recall_fscore_support(
        labels, predictions, average=None, zero_division=0
    )

    metrics = {
        # per-class
        **{f"precision_class_{i}": precision[i] for i in range(len(precision))},
        **{f"recall_class_{i}":    recall[i]    for i in range(len(recall))},
        **{f"f1_class_{i}":        f1[i]        for i in range(len(f1))},
        **{f"support_class_{i}":   support[i]   for i in range(len(support))},
        # overall
        "precision_macro": np.mean(precision),
        "recall_macro":    np.mean(recall),
        "f1_macro":        np.mean(f1),
        "accuracy":        (predictions == labels).mean(),
    }

    return metrics

# Training

## Training Parameters

In [ ]:
# hyperparameters
lr = 2e-5
batch_size = 16
num_epochs = 8

output_dir="./model_output_dataset_f"

training_args = TrainingArguments(
    seed = seed,
    output_dir=output_dir,       
    num_train_epochs=num_epochs,   
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size,   
    learning_rate=lr,             
    warmup_ratio=0.1,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",       
    load_best_model_at_end=True,  
    metric_for_best_model="f1",  
    greater_is_better=True   
)

## Run trainer

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
print("Starting training...")

try:
    train_results = trainer.train()
except Exception as e:
    print("Training failed:", e)
    raise

print("Training complete. Saving model...")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.620400,0.512544,0.743478,0.666667,0.973913,0.791519
2,0.416000,0.428950,0.826087,0.755102,0.965217,0.847328
3,0.306900,0.361369,0.852174,0.775510,0.991304,0.870229
4,0.240400,0.330381,0.865217,0.787671,1.000000,0.881226
5,0.196700,0.263788,0.900000,0.838235,0.991304,0.908367
6,0.151100,0.230804,0.921739,0.864662,1.000000,0.927419
7,0.131000,0.236226,0.926087,0.876923,0.991304,0.930612
8,0.125900,0.259122,0.913043,0.857143,0.991304,0.919355


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Training complete. Saving model...


('./model_output_dataset_f\\tokenizer_config.json',
 './model_output_dataset_f\\special_tokens_map.json',
 './model_output_dataset_f\\vocab.txt',
 './model_output_dataset_f\\added_tokens.json')

# Evaluate

In [ ]:
print("Evaluating model...")

# get metrics only from validation
val_results = trainer.evaluate(eval_dataset=val_dataset)
print("Validation F1:", round(val_results["eval_f1"], 3))

# get predictions and labels for validation
val_preds_output = trainer.predict(val_dataset)
val_logits = val_preds_output.predictions
val_labels = val_preds_output.label_ids
val_preds = np.argmax(val_logits, axis=1)

# print confusion matrix and classification report for validation
print("Validation Confusion Matrix:\n", confusion_matrix(val_labels, val_preds))
print("\nValidation Classification Report:\n", classification_report(val_labels, val_preds, zero_division=0, digits=4))


# same for test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test F1:", round(test_results["eval_f1"], 3))

test_preds_output = trainer.predict(test_dataset)
test_logits = test_preds_output.predictions
test_labels = test_preds_output.label_ids
test_preds = np.argmax(test_logits, axis=1)

print("Test Confusion Matrix:\n", confusion_matrix(test_labels, test_preds))
print("\nTest Classification Report:\n", classification_report(test_labels, test_preds, zero_division=0, digits=4))


Evaluating model...


Validation F1: 0.931
Test F1: 0.95


# Test cases

## Load trained model

In [16]:
model_path = output_dir
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# shorten text helper function
def shorten(text, max_len=40):
    if len(text) <= max_len:
        return text
    return text[:max_len-3] + "..."

## My Test Cases

In [ ]:
test_cases = [
    # neutral
    ("Today the weather is beautiful", "Heute ist das Wetter schön", 0), 
    ("I like this guy.", "Ich mag diesen Kerl.", 0),
    ("How are you?", "Wie geht es dir?", 0),
    ("The girls went hiking.", "Die Mädchen gingen wandern.", 0),

    # neutral with gendered subjects
    ("My mother is an engineer.", "Meine Mutter ist Ingenieurin.", 0),
    ("Is this boy a good artist?", "Ist dieser Junge ein guter Künstler?", 0),
    ("I am living with my sister, who is also my best friend", "Ich lebe mit meiner Schwester, die auch meine beste Freundin ist", 0),
    ("My father was an excellent cook.", "Mein Vater war ein ausgezeichneter Koch.", 0),

    # biased
    ("Do you like our maths teacher?", "Mögen Sie unsere Mathelehrerin?", 1),
    ("The doctor was late to work today.", "Der Arzt kam heute zu spät zur Arbeit.", 1),
    ("Tomorrow the students are leaving for a class trip.", "Morgen gehen die Studenten zu einer Klassenfahrt.", 1),
    ("Engineers are great.", "Ingenieure sind großartig", 1),
    ("Engineers earn a lot of money.", "Ingenieure verdienen viel Geld.", 1),

    # test GFL
    ("Engineers are kind and hard-working", "Ingenieur:innen sind lieb und fleißig", 0),
    ("Engineers are kind and hard-working", "Ingenieure und Ingenieurinnen sind lieb und fleißig", 0),
    ("Teachers are kind and hard-working", "Lehrende sind lieb und fleißig", 0),
    ("Teachers are kind and hard-working", "Lehrer:innen sind lieb und fleißig", 0),
    ("Teachers are kind and hard-working", "Lehrerinnen und Lehrer sind lieb und fleißig", 0),
    ("Teachers are kind and hard-working", "Lehrer sind lieb und fleißig", 1),
    ("Teachers are kind and hard-working", "Lehrerinnen sind lieb und fleißig", 1),

    # job posting
    ("Google's software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact with information and one another.", "Googles Software-Ingenieure entwickeln die Technologien der nächsten Generation, die ändern, wie Milliarden von Benutzern verbinden, erkunden, und interagieren mit Informationen und einander.", 1),
    ("The ideal candidate should have a solid technical foundation with a focus on Custom agent development and Copilot integrations, strategic thinking, excellent communication skills, and the ability to collaborate within a global team.", "Der ideale Kandidat sollte über solide technische Grundlagen mit Schwerpunkt auf der Entwicklung kundenspezifischer Agenten und Copilot-Integrationen, strategisches Denken, ausgezeichnete Kommunikationsfähigkeiten und die Fähigkeit zur Zusammenarbeit in einem globalen Team verfügen.", 1),
]


In [ ]:
# convert list of test cases into a dataframe
test_df = pd.DataFrame(test_cases, columns=["english", "german", "label"])

# create BiasDataset instance from dataframe
test_dataset = BiasDataset(test_df, tokenizer)

## Run Inference on each Test Case

In [ ]:
results = []

for i in range(len(test_dataset)):
    item = test_dataset[i]
    
    # prepare inputs for model, add batch dimension and move to device
    inputs = {key: val.unsqueeze(0).to(device) for key, val in item.items() if key != "labels"}
    
    # run model in evaluation mode without gradients
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_label = torch.argmax(logits, dim=1).item()
        prob = torch.softmax(logits, dim=1)[0].cpu().numpy()
    
    # collect results
    results.append({
        "english": test_df.iloc[i]["english"],
        "german": test_df.iloc[i]["german"],
        "true_label": test_df.iloc[i]["label"],
        "predicted_label": pred_label,
        "neutral_prob": prob[0],
        "biased_prob": prob[1],
        "correct": test_df.iloc[i]["label"] == pred_label
    })


## Display Results as Table

In [ ]:
results_df = pd.DataFrame(results)

print("\nBias detection test results:")
print(results_df.to_string(index=False))

accuracy = results_df["correct"].mean()
print(f"\nModel accuracy on test cases: {accuracy:.1%}")
